In [1]:
from transformers import pipeline, Conversation, AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
model_name = "facebook/blenderbot-1B-distill"

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
0
NVIDIA GeForce RTX 2070 SUPER


In [2]:


tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)

# using Seq2Seq pipeline model=model. using Causal pipeline model=model breaks. Causal pipline model=model_name works fine but
# just for potential model customization in future, like down here with torch_dtype, using model=model better so need to use Seq2Seq
# torch_dtype=torch.float16, load_in_8bit=True REDUCES TIME FROM 12S TO 3S! confirmed via testing.
model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
        torch_dtype=torch.float16, load_in_8bit=True)


#generator = pipeline("conversational", model=model, tokenizer=tokenizer)   # takes upwards of 1 min, immeseured.
#generator = pipeline("conversational", model=model_name)     # 12 sec
generator = pipeline("conversational", model=model, tokenizer=tokenizer)   # 15sec

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [3]:
convo = Conversation("Hi, what's your name?")


In [4]:
response = generator(convo) 

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


In [5]:
print(response)

Conversation id: be55b29d-a133-4d2a-acd3-e7d3139a318d
user: Hi, what's your name?
assistant:  Hi, my name is samantha. How are you doing this fine evening?



In [6]:
convo.add_user_input("Alrighty")
convo = generator(convo)

In [7]:
convo

Conversation id: be55b29d-a133-4d2a-acd3-e7d3139a318d
user: Hi, what's your name?
assistant:  Hi, my name is samantha. How are you doing this fine evening?
user: Alrighty
assistant:  What do you like to do in your spare time? I like to watch movies and play video games.

Now to add a way for user inputs

In [ ]:
import gradio as gr

In [ ]:
inputs = []
responses = []
# inputs and responses could be combined to form a 'memory' of sorts
# how to implement backstory/key memories? need to research

def convo_bot(input, response):
    convo = Conversation(text=input, past_user_inputs=inputs, generated_responses=responses)
    convo = generator(convo)
    return convo.generated_responses[-1]

demo = gr.ChatInterface(convo_bot, title="Vanilla Chatbot", description="...")
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
